<a href="https://colab.research.google.com/github/itzpankajpanwar/Tree-transform/blob/main/TT_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 7.7MB/s 
     |████████████████████████████████| 3.3MB 55.8MB/s 
     |████████████████████████████████| 901kB 58.8MB/s 
     |████████████████████████████████| 1.2MB 7.6MB/s 


In [8]:
!git clone https://github.com/itzpankajpanwar/Tree-transform.git

Cloning into 'Tree-transform'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 2), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


Main Model 

In [17]:
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.insert(0,"/content/Tree-transform")
from attention import *
from torch.nn import CrossEntropyLoss

    
class Encoder(nn.Module):
    def __init__(self, layer, N, d_model, vocab_size, word_embed):
        super(Encoder, self).__init__()
        self.word_embed = word_embed
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, inputs, mask):
        break_probs = []
        x = self.word_embed(inputs)
        group_prob = 0.
        for layer in self.layers:
            x,group_prob,break_prob = layer(x, mask,group_prob)
            break_probs.append(break_prob)

        x = self.norm(x)
        break_probs = torch.stack(break_probs, dim=1)
        return self.proj(x),break_probs


    def masked_lm_loss(self, out, y):
        fn = CrossEntropyLoss(ignore_index=-1)
        return fn(out.view(-1, out.size()[-1]), y.view(-1))


    def next_sentence_loss(self):
        pass


class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, group_attn, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.group_attn = group_attn
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask, group_prob):
        group_prob,break_prob = self.group_attn(x, mask, group_prob)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, group_prob, mask))
        return self.sublayer[1](x, self.feed_forward), group_prob, break_prob

In [ ]:
!python3 "/content/Tree-transform/main.py" -train -model_dir model_file -num_step 60000

vocab_size: 13860
/content/Tree-transform/solver.py:44: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)
total_param_num: 50994724

start epo 0!!!!!!!!!!!!!!!!

saving!!!!
Epoch Step: 1 Loss: 9.452121 Total Time: 1.335392 Step Time: 0.922801

Epoch Step: 201 Loss: 7.142012 Total Time: 88.486306 Step Time: 87.488985

Epoch Step: 401 Loss: 6.568196 Total Time: 88.928025 Step Time: 87.925942

Epoch Step: 601 Loss: 6.376680 Total Time: 89.119741 Step Time: 88.117886


finish epo 0, time 286.171292!!!!!!!!!!!!!!!


start epo 1!!!!!!!!!!!!!!!!

